In [1]:
import pandas as pd
import numpy as np
import warnings
import itertools
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import copy
import math
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
compas_scores = pd.read_csv('../data/compas-scores-two-years.csv')
protected_attributes = ['sex','race']

# Data Cleaning
Columns removed
- columns with more than 10% of data missing

Rows removed
- recidivist flag -- is_recid -- to be -1 (when no compas case would be found)
- charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested
- ordinary traffic offenses -- those with a c_charge_degree of 'O'

In [3]:
def categorize_numerical_col(num, lim1, lim2):
    if num <= lim1:
        return 0
    elif lim1 < num <= lim2:
        return 1
    elif num > lim2:
        return 2
    else:
        raise('Invalid row')
def categorize_age(age_cat):
    if age_cat=='Less than 25':
        return 0
    elif age_cat=='25 - 45':
        return 1
    elif age_cat=='Greater than 45':
        return 2
    else:
        raise('Invalid row')

In [4]:
# Data Cleaning

# Remove NaNs
percent_missing = compas_scores.isnull().sum() * 100 / len(compas_scores)
missing_value_df = pd.DataFrame({'column_name': compas_scores.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
cols2keep_df = missing_value_df[~(missing_value_df.percent_missing>10)]
cols2keep_df_list = cols2keep_df.column_name.tolist()
compas_scores_cols_trim = compas_scores[cols2keep_df_list]
compas_scores_cols_trim_dropna = compas_scores_cols_trim.dropna()

# Apply cleaning descibed in publication of data HERE: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
compas_df = compas_scores_cols_trim_dropna[(compas_scores_cols_trim_dropna['days_b_screening_arrest']<= 30) & 
                               (compas_scores_cols_trim_dropna['days_b_screening_arrest']>= -30) &
                               (compas_scores_cols_trim_dropna['is_recid']!= -1) &
                               (compas_scores_cols_trim_dropna['c_charge_degree']!= "O") &
                               (compas_scores_cols_trim_dropna['score_text']!= 'N/A') 
                              ]

# Select columns described in https://arxiv.org/abs/2106.00772 only which are (Age, Charge Degree, Gender, Prior Counts, Length Of Stay, race)

compas_subset_df = compas_df[["sex","age","age_cat","race","priors_count.1","c_charge_degree","c_jail_in", "c_jail_out","two_year_recid"]]


# Select only African American and Caucasian
compas_subset_df = compas_subset_df[(compas_subset_df["race"]=='Caucasian') |(compas_subset_df["race"]=='African-American') ]

# Add length of stay and drop "c_jail_in", "c_jail_out"

compas_subset_df["length_stay"] = pd.to_datetime(compas_subset_df["c_jail_out"]) - pd.to_datetime(compas_subset_df['c_jail_in'])
compas_subset_df["length_stay"] = compas_subset_df["length_stay"].apply(lambda x: x.days)
compas_subset_df = compas_subset_df.drop(columns = ["c_jail_in","c_jail_out"])
compas_subset_df['length_stay'] = compas_subset_df["length_stay"].apply(categorize_numerical_col, lim1=7, lim2=90)

# Categorize prior counts according to https://arxiv.org/abs/2106.00772 
compas_subset_df['priors_count.1'] = compas_subset_df["priors_count.1"].apply(categorize_numerical_col, lim1=0, lim2=3)

# Categorize age according to https://arxiv.org/abs/2106.00772
compas_subset_df['age_cat'] = compas_subset_df["age_cat"].apply(categorize_age)

# Encode categories
race_encoder = OrdinalEncoder(dtype='int')
compas_subset_df['race']  = race_encoder.fit_transform(compas_subset_df[['race']])

sex_encoder = OrdinalEncoder(dtype='int')
compas_subset_df['sex']  = sex_encoder.fit_transform(compas_subset_df[['sex']])

charge_encoder = OrdinalEncoder(dtype='int')
compas_subset_df['c_charge_degree']  = charge_encoder.fit_transform(compas_subset_df[['c_charge_degree']])

# Create protected attribute
protected_attribute = compas_subset_df["race"]

# Target Variable
target_variable = compas_subset_df['two_year_recid']

# Feature set
feature_df = compas_subset_df.drop(['two_year_recid','race','age'], axis=1)

X_train, X_test, y_train, y_test, protected_train, protected_test =train_test_split(
    feature_df.to_numpy(), target_variable.to_numpy(), protected_attribute.to_numpy(), test_size=0.2, random_state=42)


# Implementation of Fairness Feature Selection Algorithm

In [5]:
def unique_info_array(data):
    """Get unique information from input arrays"""
    unique_list = []
    
    for idx in range(data.shape[1]):
        unique_list.append(np.unique(data[:, idx]).tolist())
    return unique_list

def unique_information_conditional(y, x_s, x_s_c, protected_attr, form=1):
    """Get unique information from input arrays with conditional probabilities taken into account"""
    
    # Using 
    # IQ(T; R1|R2) = ∑t,r1,r2 QT ,R1,R2 (t, r1, r2) log((QT |R1,R2 (t|r1,r2))/ (QT |R2 (t|r2))) 
    
    if form == 1:

        row_count     = len(y)
        col_count_y  = y.shape[1]
        col_count_xs = x_s.shape[1]
        y_xs_protected_attr_xsc = np.concatenate((y, x_s, x_s_c, protected_attr), axis=1)
        ui_array = unique_info_array(y_xs_protected_attr_xsc)
        ui_array_cat_product = list(itertools.product(*ui_array)) # compute the cartesian product of all arrays
    else:
        row_count     = len(y)
        col_count_y  = x_s.shape[1]
        col_count_xs = protected_attr.shape[1]
        y_xs_protected_attr_xsc = np.concatenate((x_s, protected_attr, y), axis=1)
        ui_array = unique_info_array(y_xs_protected_attr_xsc)
        ui_array_cat_product = list(itertools.product(*ui_array)) # compute the cartesian product of all arrays

    IQ = 0
    for array in ui_array_cat_product:
        r1_r2 = len(np.where((y_xs_protected_attr_xsc == array).all(axis=1))[0]) / row_count
        r1 = len(np.where((y == array[:col_count_y]).all(axis=1))[0]) / row_count
        r2 = len(np.where((y_xs_protected_attr_xsc[:, col_count_y: -col_count_xs] == array[
            col_count_y: -col_count_xs]).all(axis=1))[0]) / row_count

        try:
            r1_given_r2 = len(np.where((y_xs_protected_attr_xsc[:, :col_count_y] == array[:col_count_y]).all(axis=1)
                                       & (y_xs_protected_attr_xsc[:, -col_count_xs:]  == array[
                                           -col_count_xs:]).all(axis=1))[0]) / len(np.where( \
                (y_xs_protected_attr_xsc[:, -col_count_xs:] == array[-col_count_xs:]).all(axis=1))[0])
        except ZeroDivisionError:
            r1_given_r2 = 0

        if r1_r2 == 0 or r1 == 0 or r2 == 0 or r1_given_r2 == 0:
            temp = 0
        else:
            temp = r1_r2 * np.log(r1_r2 / r2) / r1_given_r2
        IQ += np.abs(temp)

    return IQ

In [6]:
def unique_information(array_1, array_2):
    """Get unique information from input arrays"""
    
    row_count          = len(array_1)
    col_count_array_1  = array_1.shape[1]
        
    features_combined = np.concatenate((array_1, array_2), axis=1)
    ui_array = unique_info_array(features_combined)
    ui_array_cat_product = list(itertools.product(*ui_array))
    
    # Using 
    # IQ(T; R1|R2) = ∑t,r1,r2 QT ,R1,R2 (t, r1, r2) log((QT |R1,R2 (t|r1,r2))/ (QT |R2 (t|r2))) 
    
    row_count          = len(array_1)
    col_count_array_1  = array_1.shape[1]
    
    IQ = 0
    for array in ui_array_cat_product:
        r1_r2 = len(np.where((features_combined == array).all(axis=1))[0]) / row_count
        r1 = len(np.where((array_1 == array[:col_count_array_1]).all(axis=1))[0]) / row_count
        r2 = len(np.where((array_2 == array[col_count_array_1:]).all(axis=1))[0]) / row_count
        
        if r1_r2 == 0 or r1 == 0 or r2 == 0:
            temp = 0
        else:
            temp = r1_r2 * np.log(r1_r2 / r1) / r1
        IQ += np.abs(temp)
    return IQ


In [7]:
def get_feature_subsets(sc):
    """
    Generate all subsets of feature set
    """
    if len(sc) <= 1:
        yield sc
        yield []
    else:
        for item in get_feature_subsets(sc[1:]):
            yield [sc[0]]+item
            yield item
            
def acc_coef(y, x_s, x_s_c, protected_attr):
    return unique_information_conditional(y, x_s, x_s_c, protected_attr)

def disc_coef(y, x_s, x_s_c, protected_attr):
    return unique_information(y, np.concatenate((x_s, protected_attr), axis=1)) * unique_information(x_s, protected_attr) * unique_information_conditional(y, x_s, x_s_c, protected_attr,form=2)

def marginal_acc_coef(y_train, X_train, protected_attr, set_tracker):
    """compute  marginal accuracy coefficient"""
    num_features = X_train.shape[1]
    feat_idx = list(range(num_features))
    feat_idx.pop(set_tracker)
    feature_subsets = [x for x in get_feature_subsets(feat_idx) if len(x) > 0]
    shapley_value =0

    for sc_idx in feature_subsets:
            coef = math.factorial(len(sc_idx)) * math.factorial(num_features - len(sc_idx) - 1) / math.factorial(num_features)

            # Compute v(T ∪ {i}) 
            idx_xs_ui = copy.deepcopy(sc_idx) # create copy of subset list
            idx_xs_ui.append(set_tracker) # append feature index
            idx_xsc_ui = list(set(list(range(num_features))).difference(set(idx_xs_ui))) # compliment of x_s
            vTU = acc_coef(y_train.reshape(-1, 1), X_train[:, idx_xs_ui], X_train[:, idx_xsc_ui], protected_attr.reshape(-1, 1))

             # Compute v(T)
            idx_xsc = list(range(num_features))
            idx_xsc.pop(set_tracker)
            idx_xsc = list(set(idx_xsc).difference(set(sc_idx)))
            vT = acc_coef(y_train.reshape(-1, 1), X_train[:, sc_idx], X_train[:, idx_xsc], protected_attr.reshape(-1, 1))

            marginal = vTU - vT
            shapley_value = shapley_value + coef * marginal
    return shapley_value


def marginal_disc_coef(y_train, X_train, protected_attr, set_tracker):
    """compute marginal discrimination coefficient"""
    num_features = X_train.shape[1]
    feat_idx = list(range(num_features))
    feat_idx.pop(set_tracker)
    feature_subsets = [x for x in get_feature_subsets(feat_idx) if len(x) > 0]
    shapley_value =0

    for sc_idx in feature_subsets:
            coef = math.factorial(len(sc_idx)) * math.factorial(num_features - len(sc_idx) - 1) / math.factorial(num_features)

            # Compute v(T ∪ {i}) 
            idx_xs_ui = copy.deepcopy(sc_idx) # create copy of subset list
            idx_xs_ui.append(set_tracker) # append feature index
            vTU = disc_coef(y_train.reshape(-1, 1), X_train[:, idx_xs_ui],X_train[:, idx_xs_ui], protected_attr.reshape(-1, 1))

             # Compute v(T)
            idx_xsc = list(range(num_features))
            idx_xsc.pop(set_tracker)
            idx_xsc = list(set(idx_xsc).difference(set(sc_idx)))
            vT = disc_coef(y_train.reshape(-1, 1), X_train[:, sc_idx], X_train[:, sc_idx], protected_attr.reshape(-1, 1))

            marginal = vTU - vT
            shapley_value = shapley_value + coef * marginal
    return shapley_value
    

In [8]:
%%time
# shapley values for accuracy and discrimination
shapley_acc = []
shapley_disc = []
for i in range(5):
    acc_i = marginal_acc_coef(y_train, X_train, protected_train, i)
    disc_i = marginal_disc_coef(y_train, X_train, protected_train, i)
    
    
    shapley_acc.append(acc_i)
    shapley_disc.append(disc_i)

# DataFrame to compare shapely values
feature_names = ["sex", "Age", "Prior Count", "Charge Degree", "Length of Stay"]
shapley_df = pd.DataFrame(list(zip(feature_names, shapley_acc, shapley_disc)),
                          columns=["Feature", "Accuracy",'Discrimation'])
shapley_df

CPU times: user 8.39 s, sys: 99.6 ms, total: 8.49 s
Wall time: 8.51 s


,Feature,Accuracy,Discrimation
0,sex,0.973917,729.645575
1,Age,1.181441,939.740547
2,Prior Count,1.229856,982.431358
3,Charge Degree,1.046473,765.737748
4,Length of Stay,1.028396,908.017124


We observe that Prior count and Age have the strongest discrimatory coefficients but also have the largest impact on the accuracy 

# Prediction model using logistic Regression

We apply a logistic regression to the feature set and observe the impact on accuracy when we eliminate an individual feature and copare this with the discriminatory impact said feature has on the overall model

In [9]:
accuracy = []
calibration = []

# Build model testing impact of each feature on model accuracy

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

black  = np.where(protected_test == 1)[0] # African American
white  = np.where(protected_test == 0)[0] # Caucasian 
accuracy.append(log_reg.score(X_test, y_test))
calibration.append(log_reg.score(X_test[black], y_test[black]) - log_reg.score(X_test[white], y_test[white]))

# Test impact of each feature on model
for i in range(X_train.shape[1]):
    features = list(range(X_train.shape[1]))
    features.pop(i)
    X_train_subset = X_train[:, features]
    X_test_subset = X_test[:, features]
    
    log_reg = LogisticRegression()
    log_reg.fit(X_train_subset, y_train)
    acc_subset = log_reg.score(X_test_subset, y_test)
    cal_subset = log_reg.score(X_test_subset[black],
                               y_test[black]) - log_reg.score(X_test_subset[white],
                                                              y_test[white])
    
    accuracy.append(acc_subset)
    calibration.append(cal_subset)
    

col_names = ["None", "Sex", "Age", "Prior Count", "Charge Degree", "Length of stay"]
accuracy = [x * 100 for x in accuracy]
calibration = [x * 100 for x in calibration]
analysis = pd.DataFrame(list(zip(col_names, accuracy, calibration)),
                          columns=["Eliminating Feature", "Accuracy (%)", "Calibration (%)"])
analysis

,Eliminating Feature,Accuracy (%),Calibration (%)
0,None,66.919431,0.627451
1,Sex,65.876777,1.191410
2,Age,63.601896,-0.515406
3,Prior Count,58.578199,6.715219
4,Charge Degree,67.014218,0.468721
5,Length of stay,66.161137,1.503268


It can be observed that eliminating Prior count results in the strongest drop in accuracy despite it's high discrimatory effect. We also see that age has a significant drop in accuracy despite its high discrimatory effect.